In [0]:
import keras
import numpy as np
from keras.applications import resnet50
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img
from keras.applications.imagenet_utils import decode_predictions
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import sys
import os

Using TensorFlow backend.


In [0]:
NUM_CLASSES = 10
IMG_SIZE = 32
PER_CORE_BATCH_SIZE = 40
BATCH_SIZE = 16


IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)

In [0]:
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()
train_images = train_images.astype('float32')
test_images = test_images.astype('float32')
train_images = train_images / 255.0
test_images = test_images / 255.0


train_labels = keras.utils.to_categorical(train_labels, NUM_CLASSES)
test_labels = keras.utils.to_categorical(test_labels, NUM_CLASSES)

In [0]:
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, InputLayer, UpSampling2D, BatchNormalization
from keras.models import Sequential
from keras import optimizers
from keras.applications.resnet50 import ResNet50
from keras.models import Model
from skimage.transform import resize

restnet = ResNet50(include_top=False, weights='imagenet', input_shape=(32,32,3))

#x = restnet.output
x = restnet.layers[-1].output
x = Flatten()(x)
x = BatchNormalization()(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.3)(x) # was .3
x = BatchNormalization()(x)
#x = Dense(64, activation='relu')(x) # was 128
#x = Dropout(0.5)(x) # was .3
#x = BatchNormalization()(x)
prediction = Dense(NUM_CLASSES, activation='softmax')(x)

model = Model(restnet.input, output=prediction)

for layer in restnet.layers:
    layer.trainable = False

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: UserWarning: Update your `Model` call to the Keras 2 API: `Model(Tensor("in..., outputs=Tensor("de...)`


In [0]:
opt = optimizers.RMSprop(lr=2e-5)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [0]:
history = model.fit(train_images, train_labels,
                    epochs=10, 
                    batch_size=8,
                    validation_data=(test_images, test_labels),
                    shuffle=True,verbose=1)

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 452s 9ms/step - loss: 1.8645 - accuracy: 0.3406 - val_loss: 3.2252 - val_accuracy: 0.1000
Epoch 2/10
50000/50000 [==============================] - 471s 9ms/step - loss: 1.7361 - accuracy: 0.3860 - val_loss: 3.5774 - val_accuracy: 0.1000
Epoch 3/10
50000/50000 [==============================] - 472s 9ms/step - loss: 1.7159 - accuracy: 0.3971 - val_loss: 3.3234 - val_accuracy: 0.1000
Epoch 4/10
50000/50000 [==============================] - 474s 9ms/step - loss: 1.7105 - accuracy: 0.3969 - val_loss: 3.7212 - val_accuracy: 0.1000
Epoch 5/10
50000/50000 [==============================] - 494s 10ms/step - loss: 1.7040 - accuracy: 0.4000 - val_loss: 3.5485 - val_accuracy: 0.1000
Epoch 6/10
50000/50000 [==============================] - 482s 10ms/step - loss: 1.7025 - accuracy: 0.4028 - val_loss: 3.6721 - val_accuracy: 0.1000
Epoch 7/10
50000/50000 [==============================] - 48

# Finetuned